# 02 - Modeling — Prophet, LSTM, Elasticidad, Q-Learning (por SKU/Línea)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# === Setup de rutas y guardado interno ===
import sys
from pathlib import Path
cwd = Path.cwd().resolve()
project_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "src").exists():
        project_root = p; break
if project_root is None:
    project_root = Path("/content/drive/MyDrive/Proyecto 4")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
from src.utils.helpers import ensure_dirs
ensure_dirs(project_root / "data/processed", project_root / "models/trained_models",
            project_root / "models/model_configs", project_root / "docs")
print(" Project root:", project_root)

 Project root: /content/drive/MyDrive/Proyecto 4


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from src.data.loaders import load_sales_data
from src.models.forecasting import forecast_demand_prophet, forecast_demand_lstm
from src.models.elasticity import price_elasticity_loglog
from src.models.qlearning import PriceEnv, QLearningAgent, simple_demand
from src.utils.helpers import ensure_dirs

sales_all = load_sales_data(project_root / 'data' / 'processed' / 'sales.csv')
SKU_OBJ = None
LINE_OBJ = 'Bebidas'  # por defecto
df = sales_all.copy()
if LINE_OBJ is not None:
    df = df[df['product_line'] == LINE_OBJ]
if SKU_OBJ is not None and 'sku' in df.columns:
    df = df[df['sku'] == SKU_OBJ]
if df.empty: raise ValueError("No hay datos para el filtro elegido. Ajusta SKU_OBJ o LINE_OBJ.")
df = df.sort_values('date')

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
agg = {c: ('sum' if c in ['sales'] else 'mean') for c in num_cols}
daily = df.groupby('date', as_index=False).agg(agg)

fcst = forecast_demand_prophet(daily, 'date', 'sales', periods=30); print(fcst.tail())
series = daily.set_index('date')['sales'].astype(float)
lstm_preds = forecast_demand_lstm(series, lookback=14, epochs=3); print('LSTM head:', lstm_preds.head().tolist())

elas = price_elasticity_loglog(df, y='sales', x='price', controls=['promo','competitor_price'])
print('Elasticidad (log-log):', elas['elasticity'])

price_grid = np.linspace(df['price'].min(), df['price'].max(), 15)
env = PriceEnv(simple_demand, price_grid)
agent = QLearningAgent(n_actions=len(price_grid))
for ep in range(300):
    s = env.reset()
    for t in range(50):
        a = agent.select_action(s)
        s2, r, _, _ = env.step(a)
        agent.update(s, a, r, s2)
best_idx = agent.Q[0].argmax()
best_price = float(price_grid[best_idx])
print('Precio óptimo discreto (simulado):', best_price)

ensure_dirs(project_root / 'models/trained_models', project_root / 'models/model_configs')
(fcst.rename(columns={'ds':'date','yhat':'forecast'})
    .to_csv(project_root / 'models/trained_models' / 'prophet_fcst.csv', index=False))
(project_root / 'models/model_configs' / 'qlearning_grid.txt').write_text('\n'.join(map(str, price_grid)))
print(' Artefactos guardados en models/')

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8xap0pzx/ym93ikj2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8xap0pzx/3a0t_1fh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=28616', 'data', 'file=/tmp/tmp8xap0pzx/ym93ikj2.json', 'init=/tmp/tmp8xap0pzx/3a0t_1fh.json', 'output', 'file=/tmp/tmp8xap0pzx/prophet_modelboc_hui2/prophet_model-20250818043000.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
04:30:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:30:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


            ds       yhat  yhat_lower  yhat_upper
124 2024-02-23  85.152821   60.127494  108.782435
125 2024-02-24  87.357182   64.149901  110.428794
126 2024-02-25  89.277048   62.891871  111.350391
127 2024-02-26  83.440576   59.745382  106.693579
128 2024-02-27  87.153027   61.191415  112.404729
LSTM head: [0.053076405078172684, 0.3250334858894348, 0.20073363184928894, 0.11084019392728806, 0.023254292085766792]
Elasticidad (log-log): -0.38962193742192763
Precio óptimo discreto (simulado): 15.16857142857143
 Artefactos guardados en models/
